In [1]:
import tensorflow as tf
from f3dasm.functions.pybenchfunction import Levy, Ackley

import autograd
import autograd.core
import autograd.numpy as np
from autograd import elementwise_grad as egrad

2022-09-30 16:24:28.676736: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
function = Levy(dimensionality=2)

In [3]:
def convert_autograd_to_tensorflow(func):#S:func is completely written in numpy autograd
    @tf.custom_gradient
    def wrapper(x):
        vjp, ans = autograd.core.make_vjp(func, x.numpy())
        def first_grad(dy):            
            @tf.custom_gradient
            def jacobian(a):
                vjp2, ans2 =  autograd.core.make_vjp(egrad(func), a.numpy())
                return ans2,vjp2 # hessian                    

            return dy* jacobian(x)  
        return ans, first_grad
    
    return wrapper

In [4]:
class Model(tf.keras.Model):

  def __init__(self, seed=None, args=None):
    super().__init__()
    self.seed = seed
    self.env = args

######################### Pixel Model Class
class PixelModel(Model):
  """
    The class for performing optimization in the input space of the functions.
    The initial parameters are chosen uniformly from [0,1] so as to ensure
        similarity across all functions
    TODO: May need to add the functionality to output denormalized bounds
  """
  def __init__(self, seed=None, args = None):
    super().__init__(seed)
    z_init = tf.random.uniform((1,args['dim']), minval = 0, maxval = 1.0)
    self.z = tf.Variable(z_init, trainable=True, dtype = tf.float32)#S:ADDED 

  def call(self, inputs=None):
    return self.z

In [5]:
model = PixelModel(None,args={'dim': 2}) # Build the model
fval = []   # To store the function values at each optimziation step   
outs = []   #Storing teh outputs of the model (normalized!!) 
tvars = model.trainable_variables

2022-09-30 16:24:31.660307: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
tvars

[<tf.Variable 'Variable:0' shape=(1, 2) dtype=float32, numpy=array([[0.5338025, 0.6210842]], dtype=float32)>]

In [20]:
func = convert_autograd_to_tensorflow(function.__call__)
# func = function.__call__

In [21]:
x = tf.cast(model(None), tf.float64)
x
func(x)

TypeError: loop of ufunc does not support argument 0 of type ArrayBox which has no callable sin method

In [11]:
func(tf.cast(model(None), tf.float64))

array([[6.98931374]])

In [17]:
with tf.GradientTape() as tape:
    tape.watch(tvars)
    logits = 0.0 + tf.cast(model(None), tf.float64)           
    loss = func(tf.reshape(logits, (function.dimensionality)))
grad = tape.gradient(loss,tvars)

AttributeError: 'numpy.ndarray' object has no attribute '_id'

In [ ]:
loss

In [ ]:
func(np.array([0.70464706, 0.03490555]))

In [ ]:
tf.reshape(logits, (2))